In [1]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import warnings
warnings.simplefilter('ignore')

# Sport Selection

In [2]:
nfl = "americanfootball_nfl"
nba = "basketball_nba"
mlb = "baseball_mlb"
ncaaf = "americanfootball_ncaaf"
afl = "aussierules_afl"
nhl = "icehockey_nhl"
nrl = "rugbyleague_nrl"
mls = "soccer_usa_mls"
soccer_brazil = "soccer_brazil_campeonato"
soccer_australia = "soccer_australia_aleague"
soccer_finland = "soccer_finland_veikkausliiga"
soccer_japan = "soccer_japan_j_league"
soccer_ireland = "soccer_league_of_ireland"
soccer_norway = "soccer_norway_eliteserien"
soccer_spain2 = "soccer_spain_segunda_division"
soccer_sweden = "soccer_sweden_allsvenskan"
ucl = "soccer_uefa_european_championship"

## Make Selection Here

In [3]:
SPORT = nba

# Queries and Calculations

In [4]:
# FUNCTIONS

# Define a function to extract the data from the "sites" column and create a new DataFrame
def extract_sites(row):
    home_team = row["teams"][0]
    away_team = row["teams"][1]
    
    site = row["sites"]
    if not site:
        return None
    # Normalize the data in the "sites" column
    sites_df = json_normalize(site)
    sites_df['last_update'] = pd.to_datetime(sites_df['last_update'], unit='s', origin='1969-12-31')
    size = len(sites_df["last_update"])
    sites_df["home_team"] = [home_team] * size
    sites_df["away_team"] = [away_team] * size
    
    def decimal_odds(odds):
        home, away = odds[0], odds[-1]
        
        if home < 0:
            home = 100 / abs(home)
        else:
            home = home / 100
        
        if away < 0:
            away = 100 / abs(away)
        else:
            away = away / 100
        return [round(home, 4), round(away, 4)]
    
    sites_df['odds'] = sites_df['odds.h2h'].apply(decimal_odds)
    sites_df["sportsbook"] = sites_df['site_nice']
    sites_df["home_odds"] = sites_df["odds"].apply(lambda x: x[0])
    sites_df["away_odds"] = sites_df["odds"].apply(lambda x: x[1])
    sites_df = sites_df[['home_team', 'away_team', 'sportsbook','home_odds', 'away_odds']]
    # Return the normalized DataFrame
    return sites_df

def find_arbitrage(df):
    max_home = df.loc[df["home_odds"].idxmax()]
    max_away = df.loc[df["away_odds"].idxmax()]
    
    return guaranteed_profit(max_home["home_odds"], max_away["away_odds"], max_home["sportsbook"], max_away["sportsbook"], max_home["home_team"], max_away["away_team"])

def guaranteed_profit(home, away, home_sportsbook, away_sportsbook, home_team, away_team):
    # Calculate the amount to bet on home to win $100
    x = round(100 / (home + 1), 2)
    # Calculate the amount to bet on away to win $100
    y = round(100 / (away + 1), 2)
    # Calculate the total amount bet
    total_bet = x + y
    # Calculate the profit
    profit = 100 - total_bet
    # If the total amount bet is more than $100, return None
    if total_bet >= 99.9:
        return None
    # Otherwise, return the amount to bet on each side and the profit
    if home >= 1:
        home = str(round(home * 100))
    elif home < 1:
        home = str(round(-100 / home))
        
    if away >= 1:
        away = str(round(away * 100))
    elif away < 1:
        away = str(round(-100 / away))
        
    return pd.DataFrame(
        {"home team": home_team,
         "away team": away_team,
         "home sportsbook": [home_sportsbook],
         "away sportsbook": [away_sportsbook],
         "home bet amount": [x],
         "away bet amount": [y],
         "home team odds": [home],
         "away team odds": [away],
         "roi": [profit]
        }
    )

In [5]:
# Set the API endpoint URL
api_url = "https://api.the-odds-api.com/v3/odds"

# Set the API request parameters
params = {
    "api_key": "f922071b55b41ce5c36770c7ba68ef4a",  # Your API key
    "sport": SPORT,  # The sport to get odds for (basketball_nba for NBA)
    "region": "us",  # The region to get odds for (us for United States)
    "mkt": "h2h",  # The market to get odds for (h2h for head-to-head)
    "oddsFormat": "american"
}

# Send the API request and get the response
response = requests.get(api_url, params=params)

# Parse the response as JSON
data = response.json()

# Print the sportsbook data
data = pd.DataFrame(data["data"])
ret = pd.DataFrame(columns = ["home team","away team","home sportsbook","away sportsbook","home bet amount","away bet amount","home team odds","away team odds", "roi"])
for _, row in data.iterrows():
    ret = ret.append(find_arbitrage(extract_sites(row)), sort=False)

# Arbitrage Opportunities

In [6]:
for i in range(len(ret)):
    row = ret.iloc[i]
    print(f"Opportunity {i+1}: {row['home team']} @ {row['away team']}")
    print(f"Bet ${row['home bet amount']:.2f} on {row['home team']} via {row['home sportsbook']} sportsbook @ {row['home team odds']} odds")
    print(f"Bet ${row['away bet amount']:.2f} on {row['away team']} via {row['away sportsbook']} sportsbook @ {row['away team odds']} odds")
    print(f"ROI = ${row.roi:.2f}\n\n")


Opportunity 1: Chicago Bulls @ Utah Jazz
Bet $81.82 on Chicago Bulls via MyBookie.ag sportsbook @ -450 odds
Bet $11.11 on Utah Jazz via Caesars sportsbook @ 800 odds
ROI = $7.07


Opportunity 2: Los Angeles Lakers @ Sacramento Kings
Bet $52.38 on Los Angeles Lakers via DraftKings sportsbook @ -110 odds
Bet $28.57 on Sacramento Kings via MyBookie.ag sportsbook @ 250 odds
ROI = $19.05


